In [ ]:
# Install required packages
!pip install tensorflow

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import pathlib

# Download Food-101 dataset
url = 'http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz'
data_dir = tf.keras.utils.get_file('food-101', origin=url, untar=True)
data_dir = pathlib.Path(data_dir)

# Extract images directory from the dataset
image_dir = data_dir / 'images'

# Check if the images directory exists
if not image_dir.exists():
    print(f"Directory {image_dir} does not exist.")
else:
    print(f"Directory {image_dir} exists and contains the following subdirectories:")
    print([item.name for item in image_dir.glob('*') if item.is_dir()])


In [ ]:
# Load dataset from directory
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    image_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    image_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Normalization layer
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


Found 101000 files belonging to 101 classes.
Using 80800 files for training.
Found 101000 files belonging to 101 classes.
Using 20200 files for validation.


In [ ]:
# Load pre-trained MobileNetV2 model + higher level layers
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Add new classification head
num_classes = len(train_ds.class_names)

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 128)               163968    
                                                                 
 dense_5 (Dense)             (None, 101)               13029     
                                                                 
Total params: 2434981 (9.29 MB)
Trainable params: 176997 (691.39 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [ ]:
# # Unfreeze the base model
# base_model.trainable = True

# # Compile the model again with a lower learning rate
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# # Fine-tune the model
# history = model.fit(
#     normalized_train_ds,
#     validation_data=normalized_val_ds,
#     epochs=5  # Fine-tuning for a few more epochs
# )


In [ ]:
# Calorie Estimation
# Example mapping of food items to calories (this should be expanded with actual data)
food_to_calories = {
    'apple_pie': 265,
    'baby_back_ribs': 300,
    # Add more mappings
}

def estimate_calories(food_item):
    return food_to_calories.get(food_item.lower(), "Calorie information not available")


In [ ]:
# Function to recognize food and estimate calories
def recognize_and_estimate(image_path):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis

    predictions = model.predict(img_array)
    predicted_class = train_ds.class_names[np.argmax(predictions)]

    calorie_estimate = estimate_calories(predicted_class)

    return predicted_class, calorie_estimate

# Example usage
# Upload an image to the Colab notebook for testing
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    predicted_food, calories = recognize_and_estimate(fn)
    print(f"Predicted Food: {predicted_food}, Estimated Calories: {calories}")


Saving images.jpg to images.jpg
1/1 [==============================] - 2s 2s/step
Predicted Food: chicken_wings, Estimated Calories: Calorie information not available
